In [1]:
### load data from torch file. Unncessary if .npy files containing the IG is already done
#import torch
# Mason
#pt = torch.load('mason_unique_results.pt', map_location='cpu').numpy()
#labels = torch.load('mason_unique_label.pt', map_location='cpu').numpy()
#sequ = torch.load('mason_unique_sequence.pt', map_location='cpu').numpy()

# Brij
#pt = torch.load('brij_short_results.pt', map_location='cpu').numpy()
#labels = torch.load('brij_short_label.pt', map_location='cpu').numpy()
#sequ = torch.load('brij_short_sequence.pt', map_location='cpu').numpy()

pt = torch.load('mason_unique_results.pt', map_location='cpu').numpy()
pt should be the same as your masonIG (probably a npy file and not a csv?)

labels = torch.load('mason_unique_label.pt', map_location='cpu').numpy()
labels should be a number with either 1 or 0, and should be in the mason_sequences_labeled.csv' file


sequ = torch.load('mason_unique_sequence.pt', map_location='cpu').numpy()
are the sequences them self and are also in the mason_sequences_labeled.csv' file. However in my code they are a one-hot encoded numpy tensor, while you already have the complete sequences. You dont need the one-hot encoding, so you need to remove the code that transforms the one-hot encoding into a string

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import seaborn as sns
#import logomaker #only to make a logo graph

### Load Integrated Gradients (IG) labels and sequences into separate numpy arrays

In [2]:
### Load data from numpy files
###Takes IG data from xyzIG.npy file and seq + label data from xyz_sequences_label.csv file


#Mason
ig_data =r"C:\Users\isaac\Desktop\MTLS Masters Program\Thesis\Brijj-Mason_data\masonIG.npy"
labeled_seq_data = r"C:\Users\isaac\Desktop\MTLS Masters Program\Thesis\Brijj-Mason_data\mason_sequences_label.csv"

labeled_seq_data = pd.read_csv(labeled_seq_data)

pt = np.load(ig_data) #load Ig data

labels = labeled_seq_data.iloc[:,1].to_numpy()
labels = np.array(labels)

sequ = labeled_seq_data.iloc[:,0].to_numpy()
sequ = np.array(sequ)

### Functions to compute PCA

In [24]:


def compute_pca(pc_no):
    '''
    Computes PCA based on number of principle components provided
    '''
    pca = PCA(n_components=pc_no)
    return pca


def pca_transformation(pt, pca, pc1, pc2):
    pca_transf = pca.fit_transform(pt.reshape(-1, 20*10))
    pca_transf = pca_transf[:, [pc1-1, pc2-1]]
    return pca_transf

    
def graph_pca(pt, labels, sequ, pc1, pc2, pca):
    '''
    Graphs PCA using IG, label and sequence numpy arrays based on selected principle components and the resulting 
    PCA transformation from the pca_transform function.
    
    
    Also returns list of rows containing binders/non-binders in separate lists for further use
    '''
    
    pca_transf = pca_transformation(pt, pca, pc1, pc2)
    
    x_axes = f'PCA{pc1} explained variance: {pca.explained_variance_ratio_[pc1-1]*100:.2f}%'
    y_axes = f'PCA{pc2} explained variance: {pca.explained_variance_ratio_[pc2-1]*100:.2f}%'

    X = pd.DataFrame(pca_transf, columns = [x_axes, y_axes])
    X['Labels'] = pd.DataFrame(labels, columns=['Labels'])['Labels'].apply(lambda x: 'Binder' if x > 0.5 else 'Non Binder')

    #plt.figure(figsize = (20,15))

    ax = sns.scatterplot(x=x_axes, y=y_axes, data=X, linewidth=0, hue='Labels', s=5)
    #ax = sns.scatterplot(x=x_axes, y=y_axes, data=x_nb, linewidth=0, s=5)

    # ax.set_axis_off()
    ax.set_frame_on(False)
    # plt.title('PCA on Integraded Gradients for sequences with implanted signals')
    plt.show()
    
    binders = X.index[X['Labels'] == 'Binder'].tolist()
    nonbinders = X.index[X['Labels'] == 'Non Binder'].tolist()
    
    return binders, nonbinders, X

    
def compute_and_graph_pca(pt, labels, sequ, pc1, pc2):
    '''
    Performs all computation steps of the PCA analysis according to the set principle components using the
    IG, labels and sequence data
    '''
        
    pca = compute_pca(pc2)
    pca_transformat = pca_transformation(pt, pca, pc1, pc2)
    
    binders_and_nonbinders = graph_pca(pt, labels, sequ, pc1, pc2, pca)
    return binders_and_nonbinders



def show_all_pc_combos(pt, labels, sequ, pc_no):
    '''
    Computes and graphs all possible principle component combinations for the set pc number based on IG, label, 
    sequence.
    
    Only for quick investigation purposes. Does not return anything
    '''
    combos = []
    pca = compute_pca(pc_no)
    
    for i in range(1, pc_no+1):
        for j in range (1, pc_no+1):
            if i !=j and (i,j) not in combos and (j,i) not in combos:
                combos.append((i,j))
                compute_and_graph_pca(pt, labels, sequ, i, j)



In [20]:
# pca = compute_pca(10)
# bvnb = graph_pca(pt, labels, sequ, 1, 9, pca)
# compute_and_graph_pca(pt, labels, sequ, 2, 3)
# show_all_pc_combos(pt, labels, sequ, 3)

### 3-D PCA graphing

In [7]:
#pip install plotly


In [6]:
import plotly.graph_objects as go

In [1]:
def graphing_3d(pt, labels, sequ, pc1, pc2, pc3, show_points = None):
        
    pca = compute_pca(pc3)
    pca_transf = pca.fit_transform(pt.reshape(-1, 20*10))
    pca_transf = pca_transf[:, [pc1-1, pc2-1, pc3-1]]

    x_axes = f'PCA{pc1} expl var: {pca.explained_variance_ratio_[pc1-1]*100:.2f}%'
    y_axes = f'PCA{pc2} expl var: {pca.explained_variance_ratio_[pc2-1]*100:.2f}%'
    z_axes = f'PCA{pc3} expl var: {pca.explained_variance_ratio_[pc3-1]*100:.2f}%'

    X = pd.DataFrame(pca_transf, columns = [x_axes, y_axes, z_axes])
    X['Labels'] = pd.DataFrame(labels, columns=['Labels'])['Labels'].apply(lambda x: 'Binder' if x > 0.5 else 'Non Binder')

    if show_points == 'binders':
        X = X[X['Labels'] == 'Binder']

    elif show_points == 'non binders':
        X = X[X['Labels'] == 'Binder']

    # Create a 3D scatter plot with Plotly
    fig = go.Figure(data=[
        go.Scatter3d(
            x=X[x_axes],
            y=X[y_axes],
            z=X[z_axes],
            mode='markers',
            marker=dict(
                size=2,
                color=X['Labels'].map({'Binder': 'red', 'Non Binder': 'blue'}),
                opacity=0.5
            )
        )
    ])

    # Set labels for each axis
    fig.update_layout(scene=dict(xaxis_title=x_axes, yaxis_title=y_axes, zaxis_title=z_axes))
    fig.update_layout(width=1200, height=800)

    # Show the interactive plot
    fig.show()

#graphing_3d(pt, labels, sequ, 1, 2, 3, 'binders')
    

def show_all_pc_combos_3d(pt, labels, sequ, pc_no, show_points = None):
    combos = set()
    pca = compute_pca(pc_no)
    
    for i in range(1, pc_no+1):
        for j in range (1, pc_no+1):
            for k in range (1, pc_no+1):
                if i !=j and i != k and j != k: 
                    current_combo = frozenset({i, j, k}) 

                    if current_combo not in combos:
                        combos.add(current_combo)
                        graphing_3d(pt, labels, sequ, i, j, k, 'binders')

# show_all_pc_combos_3d(pt, labels, sequ, 10, show_points = None)


### UMAP Reduction

In [4]:
# pip install umap-learn


In [3]:
import umap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def flatten_data(unlabeled_data):
    flattened_data = unlabeled_data.reshape(unlabeled_data.shape[0], -1)
    columns = [f'Feature-{i}' for i in range(1, flattened_data.shape[1] + 1)]
    umap_df = pd.DataFrame(flattened_data, columns=columns)
    return umap_df

# print(flatten_data(pt).head)

def flatten_labeled_data(unlabeled_data, labels):
    umap_df = flatten_data(unlabeled_data)
    umap_df['Labels'] = labels
    return umap_df

# print(flatten_labeled_data(pt, labels).head)



def umap_reduction(umap_df, n_components=2, n_neighbors=15, min_dist=0.1, metric='euclidean', random_state=42):
    reducer = umap.UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=metric,
        random_state=random_state
    )

    umap_result = reducer.fit_transform(umap_df.iloc[:, :-1])  # Assuming the last column is 'Labels'

    return pd.DataFrame(umap_result, columns=[f'UMAP-{i+1}' for i in range(n_components)])

def plot_umap(umap_result, labels):
    umap_df = pd.concat([umap_result, labels], axis=1)

    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='UMAP-1', y='UMAP-2', hue='Labels', data=umap_df, palette={'Binder': 'red', 'Non Binder': 'blue'}, s=5)
    plt.title('UMAP Dimensionality Reduction')
    plt.show()


# umap_df_l = flatten_labeled_data(pt, labels)
# umap_result_l = umap_reduction(umap_df_l)
# plot_umap(umap_result_l, umap_df_l['Labels'])

umap_df_u = flatten_data(pt)
umap_result_u = umap_reduction(umap_df_u)
plot_umap(umap_result_u, umap_df_u['Labels'])



C:\Users\isaac\AppData\Roaming\Python\Python39\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


KeyError: 'Labels'

### Extracting all data points from a given set of vertices

##### Example vertices provided by Robert

In [7]:
### Vertices provided by Robert as examples

#-0.0370, 0.0512#rechts unten
#-0.1263, 0.1838#links unten
#-0.0051, 0.1493#rechts oben
#-0.0779, 0.2759#links oben

# Mason negative extract
vertices_1 = np.asarray([(-0.1263, 0.1838),#links unten
                       (-0.0370, 0.0512),#rechts unten
                       (-0.0051, 0.1493),#rechts oben
                       (-0.0779, 0.2759)])#links oben

# Mason spike right down extract
vertices_2 = np.asarray([(-0.0735, -0.096),#links unten
                       (0.1508, -0.425),#rechts unten
                       (0.2074, -0.312),#rechts oben
                       (0.0211, -0.04)])#links oben


# Mason spike right down up
vertices_3 = np.asarray([(0.0484, -0.008),#links unten
                       (0.3635, 0.15),#rechts unten
                       (0.3703, 0.295),#rechts oben
                       (0.0513, 0.121)])#links oben

# Brij spike down
vertices_4 = np.asarray([(0.0578, -0.045),#links unten
                       (0.5968, 0.029),#rechts unten
                       (0.5942, 0.098),#rechts oben
                       (0.0628, 0.009)])#links oben


# Brij spike vertical 1
vertices_5 = np.asarray([(-0.0263, 0.089),#links unten
                       (-0.0056, 0.093),#rechts unten
                       (-0.0784, 0.604),#rechts oben
                       (-0.0997, 0.588)])#links oben

# Brij spike diagonal
vertices_6 = np.asarray([(0.1356, 0.127),#links unten
                       (0.2887, 0.264),#rechts unten
                       (0.2648, 0.298),#rechts oben
                       (0.1256, 0.147)])#links oben

vertices_selection = [vertices_1, vertices_2, vertices_3, vertices_4,vertices_5,vertices_6]

#### Function to extract the rows/positions of the data points within specified vertices 

In [24]:
### ploting selection captured in vertices
def pca_selection(pt, labels, vertices, pc1, pc2, show_graph = None):
    """
    Returns the positions of the IG sequences entries that fall within the selected vertices
    Resulting list can be of all CDRH3 located within the vertices or just the binders/non-binders
    Also returns the final complete array for QC purposes
    
    Final 'show_graph' input can be left blank, or input 'y' or 'yes' to show the resulting graph and selected points
    
    Returned selection to be used in next functions to extract and convert the sequences to one letter amino acid code
    """
    pca = compute_pca(pc2)
    pca_transf = pca_transformation(pt, pca, pc1, pc2)
    
    path = Path(vertices)
    mask = path.contains_points(pca_transf)
    
    
    x_axes = f'PCA{pc1} explained variance: {pca.explained_variance_ratio_[pc1-1]*100:.2f}%'
    y_axes = f'PCA{pc2} explained variance: {pca.explained_variance_ratio_[pc2-1]*100:.2f}%'
    X = pd.DataFrame(pca_transf, columns = [x_axes, y_axes])
    X['Labels'] = pd.DataFrame(labels, columns=['Labels'])['Labels'].apply(lambda x: 'Binder' if x > 0.5 else 'Non Binder')

    X['mask'] = mask
    
    if show_graph == 'y' or show_graph == 'yes': 
        plt.figure(figsize = (20,15))
        ax = sns.scatterplot(x=x_axes, y=y_axes, data=X, linewidth=0, hue='mask', s=5)
        ax.set_frame_on(False)
        plt.show()
        
    all_rows = [int(i) for i in X.index[mask]]
    binder_rows = [int(i) for i in X.index[mask] if X.loc[i, 'Labels'] == 'Binder']
    nonbinder_rows = [int(i) for i in X.index[mask] if X.loc[i, 'Labels'] == 'Non Binder']
    
    print(f'Number of Points Selected: {len(all_rows)}    --    Number of Binders: {len(binder_rows)}    --    Number of Non Binders: {len(nonbinder_rows)}')
    
    return all_rows, binder_rows, nonbinder_rows, X 
  
all_rows, selected_binders, selected_nonbinders, xarray = pca_selection(pt, labels, vertices_6, 1, 2)


Number of Points Selected: 36    --    Number of Binders: 11    --    Number of Non Binders: 25


#### Functions to convert IG 1-hot encoded sequences from selection of row positions

In [25]:
def one_hot_to_one_letter(integrated_gradient_sequence):
    '''
    Function to convert one-hot encoded sequence to one-letter amino acid sequence string.
    '''
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    one_letter_code = ""
    
    for block in integrated_gradient_sequence:
        index = next((i for i, value in enumerate(block) if np.any(value != 0)), -1)
        if index != -1:
            one_letter_code += amino_acids[index]
        else:
            one_letter_code += "X"  # Placeholder for unknown or padding

    return one_letter_code



def onehot_to_oneletter_selection(selection):
    '''
    Returns one-letter amino acid sequence of entries, using a list of selected rows as input
    
    Can be used as an extension of graph_pca, compute_and_graph_pca and pca_selection
    '''
    selected_cdrh3_seqs = []

    for row in selection:
        ig_entry = pt[row]
        aa_seq = one_hot_to_one_letter(ig_entry)
        selected_cdrh3_seqs.append(aa_seq)
    
    return selected_cdrh3_seqs
    
print(onehot_to_oneletter_selection(selected_binders))

['WSGAGFYEFA', 'WGSRGFYEFA', 'WLGDGFYEFA', 'WRGAGMYEFT', 'WRGAGFYEFA', 'YGAGGMYEFL', 'YSDNGFYEFA', 'YGQISHYEFQ', 'WQGPSFYEFV', 'YRGCSFYEFT', 'YTDCSLYEFK']


### Function to iterate through list of vertices and export the resulting list into separate txt files